<a href="https://colab.research.google.com/github/SithmiDesilva/OralCancerEarlyDetection-DSGP/blob/Prediction-of-textual-data-II/Text_Data_Analysis_II_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Impport the dependencies
import numpy as np  #for computational calculations
import pandas as pd   # for manipulate dataframes
from sklearn.model_selection import train_test_split  # split the dataset into test train data
from sklearn.metrics import accuracy_score  # evaluate the models
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

In [2]:
# Load the dataset
data=pd.read_csv("/content/drive/MyDrive/DSGP/Dataset_with_Predicted_Symptoms (1).csv")

In [3]:
# Data Exploration
## First 5 rows of the dataset
data.head()

,public_id,lesion_id,patient_id,path,localization,larger_size,tobacco_use,alcohol_consumption,sun_exposure,gender,skin_color,age_group,diagnosis,dysplasia_severity,TaskII,TaskIII,TaskIV,symptoms,cancer_or_non_cancer
0,0,50,48,0000.png,Tongue,2.0,Yes,No,No,M,Black,2,OSCC,NaN,OSCC,Presence,OSCC,"Difficulty swallowing, Tongue pain, Chronic ir...",Cancer
1,1,50,48,0001.png,Tongue,2.0,Yes,No,No,M,Black,2,OSCC,NaN,OSCC,Presence,OSCC,"Difficulty swallowing, Tongue pain, Chronic ir...",Cancer
2,2,41,39,0002.png,Tongue,2.5,Yes,Former,Yes,M,White,1,OSCC,NaN,OSCC,Presence,OSCC,"Difficulty swallowing, Tongue pain, Chronic ir...",Cancer
3,3,41,39,0003.png,Tongue,2.5,Yes,Former,Yes,M,White,1,OSCC,NaN,OSCC,Presence,OSCC,"Difficulty swallowing, Tongue pain, Chronic ir...",Cancer
4,4,87,81,0004.png,Lip,1.5,Former,Yes,Yes,F,White,2,OSCC,NaN,OSCC,Presence,OSCC,"Ulceration on lip, Persistent soreness, Chroni...",Cancer


In [4]:
# Last 5 rows of the dataset
data.tail()

,public_id,lesion_id,patient_id,path,localization,larger_size,tobacco_use,alcohol_consumption,sun_exposure,gender,skin_color,age_group,diagnosis,dysplasia_severity,TaskII,TaskIII,TaskIV,symptoms,cancer_or_non_cancer
2232,2232,92,65,2232.png,Gingiva,1.8,Former,Former,Yes,F,Asian,3,Precancerous,Mild,Leukoplakia,Absence,Absence,NaN,Non-Cancer
2233,2233,71,31,2233.png,Buccal mucosa,2.1,Yes,Yes,No,M,Hispanic,5,OSCC,Severe,Leukoplakia,Absence,Dysplasia,"Chronic irritation, Persistent cough, Non-heal...",Cancer
2234,2234,48,87,2234.png,Cheek,1.8,No,No,Yes,F,Brown,3,OSCC,Moderate,Presence,Benign,Absence,"Non-healing ulcer, Lump in mouth, Difficulty s...",Cancer
2235,2235,51,73,2235.png,Floor of mouth,2.3,No,No,No,M,Brown,3,Benign,Severe,Presence,OSCC,Benign,NaN,Non-Cancer
2236,2236,88,92,2236.png,Cheek,2.5,Yes,Former,No,F,Asian,4,Leukoplakia without dysplasia,Moderate,OSCC,Presence,Presence,"Chronic irritation, Persistent cough",Non-Cancer


In [5]:
# Sanitory check
data.shape

(2237, 19)

In [6]:
# More details about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237 entries, 0 to 2236
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   public_id             2237 non-null   int64  
 1   lesion_id             2237 non-null   int64  
 2   patient_id            2237 non-null   int64  
 3   path                  2237 non-null   object 
 4   localization          2237 non-null   object 
 5   larger_size           2237 non-null   float64
 6   tobacco_use           2237 non-null   object 
 7   alcohol_consumption   2237 non-null   object 
 8   sun_exposure          2237 non-null   object 
 9   gender                2237 non-null   object 
 10  skin_color            2237 non-null   object 
 11  age_group             2237 non-null   int64  
 12  diagnosis             2237 non-null   object 
 13  dysplasia_severity    1571 non-null   object 
 14  TaskII                2237 non-null   object 
 15  TaskIII              

In [7]:
print(data.columns)

Index(['public_id', 'lesion_id', 'patient_id', 'path', 'localization',
       'larger_size', 'tobacco_use', 'alcohol_consumption', 'sun_exposure',
       'gender', 'skin_color', 'age_group', 'diagnosis', 'dysplasia_severity',
       'TaskII', 'TaskIII', 'TaskIV', 'symptoms', 'cancer_or_non_cancer'],
      dtype='object')


In [8]:
#drop the columns
data=data.drop(['TaskII','TaskIII','TaskIV','diagnosis','dysplasia_severity','path'],axis=1)

In [9]:
# Handle missing values
data.isnull().sum()

,0
public_id,0
lesion_id,0
patient_id,0
localization,0
larger_size,0
tobacco_use,0
alcohol_consumption,0
sun_exposure,0
gender,0
skin_color,0


In [10]:
# Impute the missing values
#Fill with mode (most common value) for categorical columns

for col in ['symptoms']:
  data[col].fillna(data[col].mode()[0], inplace=True)

<ipython-input-10-4220d01744a9>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mode()[0], inplace=True)


In [11]:
# Handle duplicate values
data.duplicated().sum()

0

In [12]:
#Statistical Description
data.describe()

,public_id,lesion_id,patient_id,larger_size,age_group
count,2237.000000,2237.000000,2237.000000,2237.000000,2237.000000
mean,1118.000000,62.080912,55.187751,2.176933,2.426911
std,645.910598,26.806605,25.483878,0.959472,1.337384
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,559.000000,43.000000,36.000000,1.500000,1.000000
50%,1118.000000,62.000000,52.000000,2.100000,2.000000
75%,1677.000000,82.000000,74.000000,2.700000,3.000000
max,2236.000000,216.000000,183.000000,6.000000,5.000000


In [13]:
#Encoding Data
## Label encoding binary categorical data
### Male-->1 and Female-->0
label_encoder=LabelEncoder()
for col in ['gender','cancer_or_non_cancer','tobacco_use','alcohol_consumption','sun_exposure','skin_color','symptoms','localization']:
  data[col]=label_encoder.fit_transform(data[col])




In [14]:
data.head()

,public_id,lesion_id,patient_id,localization,larger_size,tobacco_use,alcohol_consumption,sun_exposure,gender,skin_color,age_group,symptoms,cancer_or_non_cancer
0,0,50,48,8,2.0,3,1,0,1,1,2,4,0
1,1,50,48,8,2.0,3,1,0,1,1,2,4,0
2,2,41,39,8,2.5,3,0,2,1,5,1,4,0
3,3,41,39,8,2.5,3,0,2,1,5,1,4,0
4,4,87,81,6,1.5,0,3,2,0,5,2,9,0


In [15]:
#Scale numerical columns
scaler=MinMaxScaler()
for col in ['larger_size']:
  data[col]=scaler.fit_transform(data[[col]])


In [16]:
# Checking the distribution of target variablas
data["cancer_or_non_cancer"].value_counts() ##"value_counts" gives the count of 1 and 0s

,count
cancer_or_non_cancer,
1,1633
0,604


In [17]:
# Split features and target
X = data.drop('cancer_or_non_cancer', axis=1)
y = data['cancer_or_non_cancer']

In [18]:
print(X [:19])

    public_id  lesion_id  patient_id  localization  larger_size  tobacco_use  \
0           0         50          48             8     0.333333            3   
1           1         50          48             8     0.333333            3   
2           2         41          39             8     0.416667            3   
3           3         41          39             8     0.416667            3   
4           4         87          81             6     0.250000            0   
5           5         87          81             6     0.250000            0   
6           6        103          78             8     0.066667            1   
7           7        103          78             8     0.066667            1   
8           8        103          78             8     0.066667            1   
9           9        103          78             8     0.066667            1   
10         10        101          14             3     0.500000            3   
11         11        101          14    

In [19]:
print(y [:19])

0     0
1     0
2     0
3     0
4     0
5     0
6     1
7     1
8     1
9     1
10    0
11    0
12    0
13    0
14    1
15    1
16    1
17    1
18    0
Name: cancer_or_non_cancer, dtype: int64


In [20]:
print(data['age_group'][:19])

0     2
1     2
2     1
3     1
4     2
5     2
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    2
15    2
16    2
17    2
18    2
Name: age_group, dtype: int64


In [21]:
print(data['sun_exposure'].unique)

<bound method Series.unique of 0       0
1       0
2       2
3       2
4       2
       ..
2232    2
2233    0
2234    2
2235    0
2236    0
Name: sun_exposure, Length: 2237, dtype: int64>


In [22]:
# Split into train and test sets before applying SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to training data for balancing
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [23]:
# Check new class distribution
print("Before SMOTE:", y_train.value_counts())
print("After SMOTE:", pd.Series(y_train_resampled).value_counts())

Before SMOTE: cancer_or_non_cancer
1    1306
0     483
Name: count, dtype: int64
After SMOTE: cancer_or_non_cancer
1    1306
0    1306
Name: count, dtype: int64


In [24]:
print(X.shape,X_train_resampled.shape,X_test.shape)

(2237, 12) (2612, 12) (448, 12)


##Logistic Regression

In [25]:
##MOdel Training
##Logistic Regression

model=LogisticRegression()

In [26]:
##Training the logisticRegression model using training data
model.fit(X_train_resampled,y_train_resampled)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [27]:
#cross validation for training data
from sklearn.model_selection import cross_val_score
scores=cross_val_score(model,X_train_resampled,y_train_resampled,cv=10)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [28]:
###Accuracy on the training data
X_train_prediction=model.predict(X_train_resampled)
training_data_accuracy=accuracy_score(X_train_prediction,y_train_resampled)

In [29]:
print("Accuracy on training data:",training_data_accuracy)

Accuracy on training data: 0.7323889739663093


In [30]:
###Accuracy on test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,y_test)

In [31]:
print('Accuracy on testing data:',test_data_accuracy)

Accuracy on testing data: 0.7633928571428571


In [32]:
#Building a predictive System
input_data=([0,50,48,8,0.3,3,1,0,1,1,2,4])

# change the input data into a numpy array
input_data_as_numpy_array=np.asarray(input_data)

#reshape the numpy array as we are predicting for only one instance
input_data_reshaped=input_data_as_numpy_array.reshape(1,-1)

prediction=model.predict(input_data_reshaped)
print(prediction)

if(prediction[0]==1):
  print("The person does not prone to oral cancer")
else:
  print("The person prone to oral cancer")

[0]
The person prone to oral cancer


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


##Random Forest

In [33]:
from sklearn.ensemble import RandomForestClassifier

randomforest_classifier=RandomForestClassifier(n_estimators=100)
score=cross_val_score(randomforest_classifier,X_train_resampled,y_train_resampled,cv=10)




In [34]:
print(score)

[0.99236641 0.98473282 1.         1.         0.99233716 0.97701149
 1.         0.99233716 0.99233716 0.99233716]


In [35]:
#Accuracy of the training data
randomforest_classifier.fit(X_train_resampled,y_train_resampled)
X_train_prediction=randomforest_classifier.predict(X_train_resampled)
print(accuracy_score(X_train_prediction,y_train_resampled))

1.0


In [36]:
#Accuracy of the testing data
X_test_prediction=randomforest_classifier.predict(X_test)
print(accuracy_score(X_test_prediction,y_test))

1.0


In [37]:
#Building a predictive system
input_data_1=([0,50,48,8,0.3,3,1,0,1,1,2,4])

#change the input data into a numpy array
input_data_1_as_numpy_array=np.asarray(input_data_1)

#reshape the numpy array as we are predicting for only one instance
input_data_1_reshaped=input_data_1_as_numpy_array.reshape(1,-1)

prediction=randomforest_classifier.predict(input_data_1_reshaped)
print(prediction)

if(prediction[0]==1):
  print("The person does not prone to oral cancer")
else:
  print("The person prone to oral cancer")

[0]
The person prone to oral cancer


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
